In [45]:
import tensorflow as tf

print(f'tensorflow version {tf.__version__}')

from imblearn.over_sampling import SMOTE
from collections import Counter
from numpy import * 

import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

tensorflow version 2.9.1


# Loading the Data & Smote

In [46]:
def dataLoadSm():
    test = pd.read_csv('Real_Tomorrow/test_real_tom_target.csv')
    train = pd.read_csv('Real_Tomorrow/training_real_tom_target.csv')
    val = pd.read_csv('Real_Tomorrow/validation_real_tom_target.csv')
    print(train.head())
    
    #Split data
    x_train = train[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******training features******")
    print (x_train)
    y_train = train[['Real_tom_lsm']]
    print ("******training target******")
    print (y_train)
    
    x_test = test[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******test features******")
    print (x_test)
    y_test = test[['Real_tom_lsm']]
    print ("******test target******")
    print (y_test)
    
    x_val = val[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******validation features******")
    print (x_val)
    y_val = val[['Real_tom_lsm']]
    print ("******validation target******")
    print (y_val)
    
    print('Before Smote')

    dff = y_train
    M = y_train.to_numpy()
    # summarize dataset
    classes = unique(M)
    print(classes)
    total = len(M)
    for c in classes:
        n_examples = len(M[M==c])
        percent = n_examples / total * 100
        print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))
    
    
    smt = SMOTE()
    
    X_train_sm, Y_train_sm = smt.fit_resample(x_train, y_train)
    
    print('After SMOTE')
    dff = Y_train_sm
    M = Y_train_sm.to_numpy()
    # summarize dataset
    classes = unique(M)
    print(classes)
    total = len(M)
    for c in classes:
        n_examples = len(M[M==c])
        percent = n_examples / total * 100
        print('> Class=%d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))
        
    return X_train_sm, Y_train_sm, x_val, y_val, x_test, y_test

def dataLoad():
    test = pd.read_csv('Real_Tomorrow/test_real_tom_target.csv')
    train = pd.read_csv('Real_Tomorrow/training_real_tom_target.csv')
    val = pd.read_csv('Real_Tomorrow/validation_real_tom_target.csv')
    print(train.head())
    
    #Split data
    x_train = train[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******training features******")
    #print (x_train)
    y_train = train[['Real_tom_lsm']]
    print ("******training target******")
    #print (y_train)
    
    x_test = test[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******test features******")
    #print (x_test)
    y_test = test[['Real_tom_lsm']]
    print ("******test target******")
    #print (y_test)
    
    x_val = val[[ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']]
    print ("******validation features******")
    #print (x_val)
    y_val = val[['Real_tom_lsm']]
    print ("******validation target******")
    #print (y_val)
    
        
    return x_train, y_train, x_val, y_val, x_test, y_test

In [47]:
x_train_sm, y_train_sm, x_val, y_val, x_test, y_test = dataLoadSm()
x_train, y_train, x_val, y_val, x_test, y_test = dataLoad()


   Unnamed: 0        time  latitude  longitude        vo          r     u_200  \
0           0  1980-01-01       0.0       20.0  0.000007  80.761185  1.909660   
1           1  1980-01-01       0.0       22.5  0.000004  80.703650  1.165733   
2           2  1980-01-01       0.0       25.0  0.000007  78.231514 -1.311676   
3           3  1980-01-01       0.0       27.5  0.000010  79.631010 -3.777573   
4           4  1980-01-01       0.0       30.0  0.000010  71.573875 -5.734505   

      u_850     v_200     v_850        ttr  sst  lsm  Real_tom_lsm  
0 -3.323872  1.687164 -1.823624 -247.54074  0.0  0.0           0.0  
1 -2.844494  1.060593 -1.991425 -240.00592  0.0  0.0           0.0  
2 -2.125244  3.280617 -1.931789 -223.76889  0.0  0.0           0.0  
3 -1.122395  5.743889 -1.243538 -235.55556  0.0  0.0           0.0  
4 -1.362953  6.514030 -0.954163 -254.03260  0.0  0.0           0.0  
******training features******
               vo          r      u_200     u_850      v_200     v_85

# Standardization

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#scaled_x_training = scaler.fit_transform(x_training)
#df = pd.DataFrame(StandardScaler().fit_transform(x_training))

X_train_stand = x_train_sm.copy() #smote
X_valid_stand = x_val.copy() #smote
X_test_stand = x_test.copy() #smote

#not smote
X_train_scaled = x_train.copy() 
X_valid_scaled = x_val.copy()
X_test_scaled = x_test.copy()

num_cols = [ 'vo', 'r', 'u_200', 'u_850', 'v_200','v_850', 'ttr','sst']

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_stand[[i]])
    
    # transform the training data column
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    
    # transform the training data column
    X_valid_stand[i] = scale.transform(X_valid_stand[[i]])   

    # transform the testing data column
    X_test_stand[i] = scale.transform(X_test_stand[[i]])
    
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_scaled[[i]])
    
    # transform the training data column
    X_train_scaled[i] = scale.transform(X_train_scaled[[i]])
    
    # transform the training data column
    X_valid_scaled[i] = scale.transform(X_valid_scaled[[i]])   

    # transform the testing data column
    X_test_scaled[i] = scale.transform(X_test_scaled [[i]])

# ExtraTreesClassifier

In [49]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import log_loss
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

# get a list of models to evaluate
def get_models():
    models = dict()
    #n_trees = [10, 50, 100, 500, 1000, 5000]
    n_trees = [1]
    min_samples_leaf = [1]
    #    min_samples_leaf = [1, 10, 50, 100, 500, 1000, 5000]
    max_features= ['sqrt', 'log2', None] 
    for n in n_trees:
        for l in min_samples_leaf:
            for m in max_features:
                models[str(n)+'trees_'+str(l)+'leafSamples_'+str(m)+'maxFeat'] = ExtraTreesClassifier(criterion='log_loss', n_estimators=n, min_samples_leaf=l, max_features=m)
    return models

#ExtraTreesClassifier_class_weight
def get_models_class_weight(method):
    models_class_weight = dict()
    #n_trees = [10, 50, 100, 500, 1000, 5000]
    n_trees = [1]
    min_samples_leaf = [1, 10]
    #    min_samples_leaf = [1, 10, 50, 100, 500, 1000, 5000]
    max_features= ['sqrt', 'log2', None] 
    
    if method == 'balanced':
        print("The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))")
        for n in n_trees:
            for l in min_samples_leaf:
                for m in max_features:
                    models_class_weight[str(n)+'trees_'+str(l)+'leafSamples_'+str(m)+'maxFeat'] = ExtraTreesClassifier(criterion='log_loss', n_estimators=n, min_samples_leaf=l, max_features=m, class_weight="balanced")
    elif method == 'balanced_subsample':
        print("The “balanced_subsample” mode is the same as “balanced” except that weights are computed based on the bootstrap sample for every tree grown.")
        for n in n_trees:
            for l in min_samples_leaf:
                for m in max_features:
                    models_class_weight[str(n)+'trees_'+str(l)+'leafSamples_'+str(m)+'maxFeat'] = ExtraTreesClassifier(criterion='log_loss', n_estimators=n, min_samples_leaf=l, max_features=m, class_weight="balanced_subsample")
    else:
        print('the method is not callable')
    return models_class_weight


# evaluate a given model using validation
def evaluate_model(model, x_train, y_train, x_val, y_val):
    trained_mod = model.fit(x_train, y_train)
    score = log_loss(y_val, model.predict_proba(x_val))
    conf = confusion_matrix(y_val, np.round(model.predict(x_val)))
    np.set_printoptions(precision=2)
    #print(model)  
    return score,conf

# models


In [50]:
models = get_models()
models_class_weight_balanced = get_models_class_weight('balanced')
models_class_weight_balanced_subsample = get_models_class_weight('balanced_subsample')


The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))
The “balanced_subsample” mode is the same as “balanced” except that weights are computed based on the bootstrap sample for every tree grown.


# random_forest_smote

In [51]:
print("#------------#--------------- Smote is doing ---------------#------------#")

results1, names1, confs1 = list(), list(), list()
for name1, model in models.items():
    res1 = evaluate_model(model, X_train_stand, y_train_sm.values.reshape(-1,), X_valid_scaled, y_val.values.reshape(-1,))
    print(name1)
    print(res1[0])
    print(res1[1])
    results1.append(res1[0])
    names1.append(name)
    confs1.append(res1[1])
    
print("the smote is done ---->")

import pickle
sample_list = results1
file_name = "results_smote.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the results for smote is saved on results_smote.pkl")

sample_list = names1
file_name = "names_smote.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the name for smote is saved on names_smote.pkl")


sample_list = confs1
file_name = "confs_smote.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the confs for smote is saved on confs_smote.pkl")



#------------#--------------- Smote is doing ---------------#------------#
1trees_1leafSamples_sqrtmaxFeat
2.0256602365596317
[[647450  39713]
 [   661    578]]
1trees_1leafSamples_log2maxFeat
1.9436785447149196
[[649347  37816]
 [   924    315]]
1trees_1leafSamples_NonemaxFeat
2.709961518732239
[[633665  53498]
 [   515    724]]
the smote is done ---->
the results for smote is saved on results_smote.pkl
the name for smote is saved on names_smote.pkl
the confs for smote is saved on confs_smote.pkl


In [52]:
results2, names2, confs2 = list(), list(), list()

print("#------------#--------------- class_weight_balanced is doing ---------------#------------#")

for name2, model in models_class_weight_balanced.items():
    res2 = evaluate_model(model, X_train_scaled, y_train.values.reshape(-1,), X_valid_scaled, y_val.values.reshape(-1,))
    print(name2)
    print(res2[0])
    print(res2[1])
    results2.append(res2[0])
    names2.append(name2)
    confs2.append(res2[1])
    
print("the class_weight_balanced is done ---->")

import pickle
sample_list = results2
file_name = "results_class_weight_balanced.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the results for class_weight_balanced is saved on results_class_weight_balanced.pkl")

sample_list = names2
file_name = "names_class_weight_balanced.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the name for class_weight_balanced is saved on names_class_weight_balanced.pkl")

sample_list = confs2
file_name = "confs_class_weight_balanced.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the confs for class_weight_balanced is saved on names_class_weight_balanced.pkl")

#------------#--------------- class_weight_balanced is doing ---------------#------------#
1trees_1leafSamples_sqrtmaxFeat
0.11820616033423816
[[685789   1374]
 [   982    257]]
1trees_1leafSamples_log2maxFeat
0.12257115861483185
[[685704   1459]
 [   984    255]]
1trees_1leafSamples_NonemaxFeat
0.11845702230438722
[[685782   1381]
 [   980    259]]
1trees_10leafSamples_sqrtmaxFeat
0.16186666944986647
[[650445  36718]
 [   380    859]]
1trees_10leafSamples_log2maxFeat
0.11119061840965049
[[668483  18680]
 [   450    789]]
1trees_10leafSamples_NonemaxFeat
0.08459736522736616
[[677573   9590]
 [   534    705]]
the class_weight_balanced is done ---->
the results for class_weight_balanced is saved on results_class_weight_balanced.pkl
the name for class_weight_balanced is saved on names_class_weight_balanced.pkl
the confs for class_weight_balanced is saved on names_class_weight_balanced.pkl


In [53]:
results3, names3, confs3 = list(), list(), list()
print("#------------#--------------- class_weight_balanced_subsample is doing ---------------#------------#")

for name3, model in models_class_weight_balanced_subsample.items():
    res3 = evaluate_model(model, X_train_scaled, y_train.values.reshape(-1,), X_valid_scaled, y_val.values.reshape(-1,))
    print(name3)
    print(res3[0])
    print(res3[1])
    results3.append(res3[0])
    names3.append(name3)
    confs3.append(res3[1])
    
    
print("the class_weight_balanced_subsample is done ---->")

import pickle
sample_list = results3
file_name = "results_class_weight_balanced_subsample.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the results for class_weight_balanced_subsample is saved on results_class_weight_balanced_subsample.pkl")

sample_list = names3
file_name = "names_class_weight_balanced_subsample.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the name for class_weight_balanced_subsample is saved on results_class_weight_balanced_subsample.pkl")

sample_list = confs3
file_name = "names_class_weight_balanced_subsample.pkl"

open_file = open(file_name, "wb")
pickle.dump(sample_list, open_file)
open_file.close()

print("the confs for class_weight_balanced_subsample is saved on results_class_weight_balanced_subsample.pkl")


#------------#--------------- class_weight_balanced_subsample is doing ---------------#------------#
1trees_1leafSamples_sqrtmaxFeat
0.11695185048349284
[[685805   1358]
 [   973    266]]
1trees_1leafSamples_log2maxFeat
0.11705219527155249
[[685807   1356]
 [   977    262]]
1trees_1leafSamples_NonemaxFeat
0.1174034020297612
[[685826   1337]
 [  1003    236]]
1trees_10leafSamples_sqrtmaxFeat
0.17896676603360367
[[650466  36697]
 [   413    826]]
1trees_10leafSamples_log2maxFeat
0.11982538177718811
[[666570  20593]
 [   437    802]]
1trees_10leafSamples_NonemaxFeat
0.08604680168665516
[[677242   9921]
 [   524    715]]
the class_weight_balanced_subsample is done ---->
the results for class_weight_balanced_subsample is saved on results_class_weight_balanced_subsample.pkl
the name for class_weight_balanced_subsample is saved on results_class_weight_balanced_subsample.pkl
the confs for class_weight_balanced_subsample is saved on results_class_weight_balanced_subsample.pkl


# ExtraTreesClassifier_class_weight

In [55]:
file_name = "results_class_weight_balanced_subsample.pkl"
open_file = open(file_name, "rb")
loaded_list_names = pickle.load(open_file)
open_file.close()
loaded_list_names

[0.11695185048349284,
 0.11705219527155249,
 0.1174034020297612,
 0.17896676603360367,
 0.11982538177718811,
 0.08604680168665516]